# redshift-entity-resolution-ibc-analysis

An analysis of possibly missing members in IBC data

In [2]:
import pandas as pd
from os import environ
import io
from operator import itemgetter
import logging
import numpy as np

import s3fs
import io
import boto3
import gzip
import fastparquet as fp
import awswrangler as wr
import redshift_connector
from s3fs import S3FileSystem
from fastparquet import ParquetFile
from sqlalchemy.engine import create_engine
from pandas.io.sql import SQLTable

from helpers import (
    save_dataframe_csv,
    get_training_data,
    parquet_file,
    parquet_dataframe,
    make_dataframe_db_schema,
    download_from_s3,
    save_manifest_s3,
    save_data_qa,
    save_json_s3
)

dsn = create_engine(environ["ANALYTICS"])

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

log = logging.getLogger(__name__)

## Reading csv files from from S3

Simple as pie

In [ ]:
s3 = s3fs.S3FileSystem(anon=False)

df_raw = pd.read_csv(
    's3://qh-clinicaldata-phi/raw_feed/pre_ingest/healthy_blue/y=2022/m=03/d=10/'
    'ts=134943/QUARTET_GBDFACETS_PATIENT_20220310.txt',
    delimiter="|"
)

## Redshift operations

Query data in redshift and put it into a pandas dataframe

In [40]:
sf_scope_query = """
SELECT DISTINCT
       "case".patient__birthdate__c AS dob
     , "account".name
     , "case".insurance__plans__c AS sf_plan
     , "case".most__recent__referred__date__c
     , "case".activation__channel__c
     , "case".case__closed__reason__c
     , account.patient__claims__data__c
     , account.quartet_id__c AS patient_id
FROM salesforce."case"
LEFT JOIN salesforce.account
  ON account.id = "case".account_id
WHERE "case".created_date BETWEEN '2024-01-01' AND '2025-01-01'
  AND "case".insurance__category__c NOT IN ('Medicare', 'Medicaid')
  AND account.patient__claims__data__c IS NULL
  AND (
      "case".insurance__plans__c ILIKE '%%Independence%%'
   OR "case".insurance__plans__c ILIKE '%%Amerihealth%%'
    )
"""

In [41]:
sf_scope_df = pd.read_sql(sf_scope_query, dsn)

In [42]:
sf_scope_df['first_name'] = sf_scope_df['name'].apply(lambda x: x.split()[0])
sf_scope_df['last_name'] = sf_scope_df['name'].apply(lambda x: ' '.join(x.split()[1:]))

In [ ]:
data_df = []

for i, row in sf_scope_df.iterrows():
    if True:
        dob = row['dob']
        first_name = row['first_name']
        last_name = row['last_name']
        patient_id = row['patient_id']
        sf_plan = row['sf_plan']
        query = f'''
            SELECT DISTINCT
                UPPER(mm.member_first_name) AS member_first_name
              , UPPER(mm.member_last_name) AS member_last_name
              , mm.member_dob
              , mm.member_id
              , TRUE AS found_in_member_file
              , '{patient_id}' AS patient_id
              , '{sf_plan}' AS sf_plan
              , pi.insurance_carrier
              , pi.insurance_plan
              , pi.card_ids
            FROM independence_prod.member_month AS mm
            LEFT JOIN independence_prod.patient_insurance AS pi
              ON pi.patient_insurance_quid = mm.patient_insurance_quid
            WHERE member_dob = '{dob}'
              AND member_first_name ILIKE '%%{first_name}%%'
              AND member_last_name ILIKE '%%{last_name}%%'
            '''
        row_df = pd.read_sql(query, dsn)
        if row_df.empty:
            row_df = pd.DataFrame(
                {
                    'member_first_name': first_name.upper(),
                    'member_last_name': last_name.upper(),
                    'member_dob': dob,
                    'member_id': '',
                    'patient_id': patient_id,
                    'found_in_member_file': False,
                    'sf_plan': sf_plan,
                    'insurance_carrier': '',
                    'insurance_plan': '',
                    'card_ids': ''
                }, index = [0]
            )
        data_df.append(row_df)

missing_data_df = pd.concat(data_df, ignore_index=True)

Put data into a table in redshift (connector for awswrangler is different than sqlalchemy)

In [ ]:
schema_name = 'ahmiel'
table_name = 'ibc_missing_sf_claims_members_lookup'

connector = redshift_connector.connect(
    host=environ['PGHOST'],
    database=environ['PGDATABASE'],
    user=environ['PGUSER'],
    password=environ['PGPASSWORD']
    )

In [ ]:
wr.redshift.to_sql(
        df=missing_data_df,
        table=table_name,
        schema=schema_name,
        con=connector,
        mode='overwrite',
        dtype={'value': 'VARCHAR(5)'},
        overwrite_method='drop',
        index=False,
        chunksize=1000
    )